In [ ]:
with open("pass.txt", "r") as f:
    login = !whoami
    login = login[0]
    passwd = f.read()
!echo $passwd | kinit $login

Ссылка на простые действия спарк
https://pythonru.com/biblioteki/pyspark-dlja-nachinajushhih

## Чтение данных из источника
Основной метод чтения любых источников

```df = spark.read.format(datasource_type).option(datasource_options).load(object_name)```

+ ```datasource_type``` - тип источника ("parquet", "json", "cassandra") и т. д.
+ ```datasource_options``` - опции для работы с источником (логины, пароли, адреса для подключения и т. д.)
+ ```object_name``` - имя таблицы/файла/топика/индекса

[DataframeReader](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader):
+ по умолчанию выводит схему данных
+ является трансформацией (ленивый)
+ возвращает [Dataframe](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)

### Список (неполный) поддерживаемых источников данных
+ Файлы:
  - json
  - text
  - csv
  - orc
  - parquet
  - delta
+ Базы данных
  - elasticsearch
  - cassandra
  - jdbc
  - hive
  - redis
  - mongo
+ Брокеры сообщений
  - kafka
  

**Библиотеки для работы с источниками должны быть доступны в JAVA CLASSPATH на драйвере и воркерах!**

In [ ]:
import re
import json
import datetime
from sys import argv
from pyspark import SparkConf, SparkContext
from pyspark.sql import HiveContext, SQLContext, SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from ### import vertica
from ###.sparksession_helper import get_spark_session

## Запустить спарк сессию

In [ ]:
# запуск 
spark_session = get_spark_session(template="MEDIUM_30",
                                  app_name="Answer_email",
                                  max_live_hours=10, 
                                  vrt_jdbc_support = True)

## Открыть файл CSV

In [ ]:
# открыть CSV формат spark 
df = spark_session.read.format("csv") \
        .options(header=True, inferSchema=True) \
        .load("/tmp/datasets/airport-codes.csv")

## Открыть файл Parquet

In [ ]:
df = spark.read.parquet('/data/stats/maxima/airflow_files/stations_stat')

## Открыть файл AVRO в нескольких папках

In [ ]:
root_path = config['files']['pcs']['root_path']
filenames = config['files']['pcs']['file_name']

In [ ]:
def gendates(daycnt, sevodnya=False):
    """
    Генерирует пути для чтения avro-файлов
    Args:
        daycnt: количество дней
        sevodnya: нужно ли учитывать сегодняшний день

    Returns: генератор дня, месяца, года

    """
    if sevodnya:
        td = 0
    else:
        td = 1
    for i in range(td, daycnt + 1):
        # a = datetime(2021,9,21,12) - timedelta(days=i)
        a = datetime.today() - timedelta(days=i)
        yield a.day, a.month, a.year


def genpathes(root_path, daycnt, sevodnya=False):
    """
    Генерирует пути для чтения avro-файлов
    Args:
        root_path: корневая директория
        daycnt: количество дней
        sevodnya: нужно ли учитывать сегодняшний день

    Returns: массив путей

    """
    return [path.join(root_path, f'year={year}/month={month:02}/day={day:02}/')
            for day, month, year in gendates(daycnt, sevodnya)]


def genmailpathes(root_path, filename, daycnt, sevodnya=False):
    """
    Генерирует пути для каталогов из filename внутри root_path
    Args:
        root_path: корневая директория
        filename: тип рассылки
        daycnt: количество дней
        sevodnya: нужно ли учитывать сегодняшний день

    Returns: массив путей

    """
    mailpathes = {}
    for folder in filename:
        full_path = root_path + folder
        mailpathes[folder] = genpathes(full_path, daycnt, sevodnya)
    return mailpathes

def readavro(spark_session, root_path, path2avro):
    """
    Проверяем список путей на наличие пропусков. Очищенный список подаем на чтение
    Args:
        spark_session: спарк-сессия
        root_path: базовый путь каталога с данными
        path2avro: список путей до авро-файла
    Returns: датафрейм
    """
    list2read = []
    for i in path2avro:
        list2read.append(i)
    if list2read:
        dfavro = spark_session.read \
            .option('basePath', root_path) \
            .format('avro').load(list2read)
        return dfavro
    else:
        logging.warning('Часть данных отсутствует')


def prepareevents(spark_session, root_path, dict_pathes):
    """
    Обрабатываем датафрейм для каталога 'mail_events' (событие создания рассылки)
    Args:
        spark_session: спарк-сессия
        root_path: базовый путь каталога с данными
        dict_pathes: словарь нагенеренных путей к каталогу

    Returns: готовый датафрейм
    """
    df = readavro(spark_session, root_path, dict_pathes['mail_event'])
    if df is not None:
        return df \
            .select('event_uuid', 'event_time', col('contact_to').alias('email'),
                    'topic', 'mailing_id', 'campaign_id') \
            .where((col('topic') != '') & (col('stream_type') == 'email'))
    else:
        schema = StructType([
            StructField('event_uuid', StringType(), True),
            StructField('event_time', LongType(), True),
            StructField('email', StringType(), True),
            StructField('topic', StringType(), True),
            StructField('mailing_id', LongType(), True),
            StructField('campaign_id', LongType(), True)
        ])
        return spark_session.createDataFrame([], schema=schema)

In [ ]:
dicts = genmailpathes(root_path, filenames, 28, False)
# можно удалить из словаря даты которой еще нет в списке
#dicts['mail_open'].remove('/data/stats/pcs/landing/mail_open/year=2021/month=09/day=05/')

In [ ]:
event = spark.read \
            .option('basePath', root_path) \
            .format('avro').load(dicts['mail_event'])

## Открыть файл Excel 

In [ ]:
import pandas as pd
# исходный файл сохранила в обычном формате xlsx
df_orig = pd.read_excel('file.xlsx', engine='openpyxl', header=None)

In [ ]:
#  open in spark session file xlsx
df_spark = spark_session.createDataFrame(df_orig.astype(str))

## Записать в  файл

In [ ]:
## сохраняю результат в csv.
df.repartition(1).write.mode('overwrite').csv('./file.csv', encoding='windows-1251', header=True)

## Запускаем Vertica

In [ ]:
# запускаем VERTICA для открытия таблиц
KERBEROS_VERTICA = {
    "user": login,
    "password": passwd,
    "db": "verticadb",
    "hdfs_url": "hdfs://nameservice1/user/vertica-udrvs/", 
    "url": "jdbc:vertica://udrvs-ver-1x-cluster.data.corp:5433/verticadb",
    "host": "udrvs-ver-1x-cluster.data.corp",
}

In [ ]:
elk_main = vertica.get_table_via_jdbc(tablename='main', scheme='data', \
                                      spark_session=spark_session, opts=KERBEROS_VERTICA)\
.select('ID', 'EMAIL', 'uuid') # выбираем колонки для отображения
# tablename - имя таблицы
# scheme - имя схемы
# spark_session - спарк-сессия
# opts - конфигурация соединения (см. выше)

Метод `show` выводит часть датафрейма в консоль

In [ ]:
# вывод данных 
df.show(10, False)
# Дополнительный параметр vertical=True выведет каждую строку данных по строчно ввиду колонка|значение
df.show(n=1, truncate=False, vertical=True)
df.printSchema()

In [ ]:
# показать таблицу с Null в колонке col1
from pyspark.sql import functions as F
df.where(F.col('col1').isNull()).show()

Метод `select` позволяет выбрать существующие (а также создать новые) колонки из датафрейма

In [ ]:
df.select("continent", "country").show(10, False)
df.localCheckpoint().select("continent", "country").explain()

In [ ]:
# Удалим дубликаты. По умолчанию метод dropDuplicates удаляет дубликаты строк, у которых ВСЕ колонки совпадают
df.dropDuplicates(subset=["name"]).show(10, False)
df.localCheckpoint().dropDuplicates(subset=["name"]).explain()
# Метод .na.drop удаляет СТРОКИ, в которых отсутствует часть данных. Параметр how="all" означает,
# что будут удалены строки, у которых ВСЕ колонки null
df.dropDuplicates().na.drop(how="all").show(10, False)
df.localCheckpoint().dropDuplicates().na.drop(how="all").explain()

In [ ]:
# Метод .na.fill заполняет null. Для работы этого метода требуется словарь с изменениями
fill_dict = {'continent': 'n/a', 'population': 0 }

df.dropDuplicates().na.drop(how="all").na.fill(fill_dict).show(10, False)
df.localCheckpoint() \
    .dropDuplicates() \
    .na.drop(how="all") \
    .na.fill(fill_dict).explain(extended=True)

In [ ]:
# Метод .na.replace заменяет данные в колонках. Для его работы требуется словарь с заменами
replace_dict = {"Rossiya": "Russia"}

df.dropDuplicates().na.drop("all").na.fill(fill_dict).na.replace(replace_dict).show(10, False)

In [ ]:
# Подготовим базовый агрегат. По умолчанию имена колонок принимают неудобные названия
from pyspark.sql.functions import count, sum

agg = clean_data.groupBy("continent").agg(count("*"), sum(col("population")))
agg.show(10, False)

In [ ]:
# Метод alias позволяет переименовывать колонки
from pyspark.sql.functions import count, sum, lower

pop_count = count("*").alias("city_count")
pop_sum = sum(col("population")).alias("population_sum")

agg = clean_data \
            .groupBy("continent") \
            .agg(pop_count, pop_sum) \
            .withColumn("continent", lower(col("continent")))

In [ ]:
# нижний регистр
df.withColumn(column, F.lower(col(column)))

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
# Выводим нужные колонки вариант SQL
df.select('uuid','vehicle_brand','vehicle_year').show(5)
# Выводим нужные колонки вариант DataFrame
df.select(df['uuid'],df['vehicle_brand'],df['vehicle_year']).show(5)
# Удобнее и логичнее использовать col() для составления условий фильтрации
df.select(F.col('uuid'),F.col('vehicle_brand'),F.col('vehicle_year')).show(5)

In [ ]:
# Выводим нужные колонки вариант SQL
df\
    .select('uuid','vehicle_brand','vehicle_year')\
    .filter(F.col("vehicle_brand") == 'Бмв 328i Хdrivе')\
    .filter(F.col("vehicle_year") == '2013')\
    .show(5)

In [ ]:
df.count()

In [ ]:
df.groupBy("vehicle_brand").count().show(5)
df.groupBy("vehicle_brand").count().orderBy("count").show(5)
df.groupBy("vehicle_brand").count().orderBy(F.col("count").desc()).show(5)

In [ ]:
# Разделение колонки на две по знаку
from pyspark.sql.functions import split
df.select(split(F.col('email'), '@')[0].alias('Мыло')
# Разделение колонки на две по знаку для всей таблицы
df_spark.withColumn('ДобавочныйНомер',split(df_spark[col_14],'доб').getItem(1)).show(100,False,True)
# вариант создания функции для резделения
split_col = pyspark.sql.functions.split(df['my_str_col'], '-')
df = df.withColumn('NAME1', split_col.getItem(0))
df = df.withColumn('NAME2', split_col.getItem(1))

In [ ]:
# вариант дублирования колонки 'column'
df.withColumn('NAME1',col('column')).show(100,False,True)

In [ ]:
# выбираем данные с индекса 5 по 7 и записываем в новую колонку
df.select(col('column').substr(5, 7).alias("NewNameColumn")).show(5,False,True)
df.withColumn(column, col(column).substr(0, 10))

In [ ]:
# обрезает и выводит с 1 позиции до 12
df.select(col('column').substr(1, 12)).show()

In [ ]:
# удаляем в начале только символ 7 на пустоту
.withColumn(col_14,F.regexp_replace(col_14, '^[7]',''))\
# выбираем начало на цифры в списке и заменяем всё значение на 'Нет'
.withColumn(col_14, F.when(col(col_14).rlike("^[0123567]"), 'Нет').otherwise(col(col_14)))

In [ ]:
# filter AND (&), OR (|) и NOT (~)
df.filter( (col('data') >= lit('2020-01-01')) & (col('data') <= lit('2020-01-31')) ).show(5)
# Between
df.filter(data.adjusted.between(100.0, 500.0)).show()
# Он возвращает 0 или 1 в зависимости от заданного условия. В приведенном ниже примере показано, 
#как выбрать такие цены на момент открытия и закрытия торгов, при которых скорректированная цена была больше или равна 200
data.select('open','close',f.when(data.adjusted >= 200.0, 1).otherwise(0)).show(5)

In [ ]:
# убираем дубли в строке
from pyspark.sql.functions import udf
remove = udf(lambda row: (sorted(set(row.split(' ')), key=row.index)), returnType=StringType())
df_join = udf(lambda row: (" ".join(row)), returnType=StringType())
df_spark.withColumn(col_4, remove(col_4)).select(col_4)\
    .withColumn(col_4, df_join(col_4)).select(col_4)\
    .filter(col(col_4).rlike('Барсков')).show(5,False)

In [ ]:
df.where(F.col('mailing_id')==139543)

In [ ]:
# указываем колонку для заполения null и заменям значением unknown
df.na.fill('UNKNOWN', "column")

In [ ]:
df.groupBy('open','click').count().show()

# length

In [2]:
# выводим кол-во символов в каждой строчке и записываем в новую колонку
df.select(length('column').alias('length')).show(100,False,True)
# выводим длину строчек в колонке
df.select(length('column').alias('length')).distinct().show()
# выводим длину строчек
.select('column').filter(length('column')==6).distinct().show()
# выводим варианты длины строки и кол-во
.select(length('column').alias('Длина')).groupBy('Длина').count().show()
# выбираем длину цифр меньше 9 и больше 11 и заменяем всё значение на 'Нет' все остальные значения оставляем
.withColumn(col_14, F.when(length(col_14) < 9, 'Нету').when(length(col_14) > 11, 'Нет').otherwise(col(col_14)))

## join

In [ ]:
df = ev \
        .join(opend.select('uuid').withColumn('open', F.lit(1)), 'uuid', 'left') \
        .join(send.select('uuid').withColumn('send', F.lit(1)), 'uuid', 'left') \
        .join(undel.select('uuid').withColumn('undel', F.lit(1)), 'uuid', 'left') \
        .join(click.select('uuid').withColumn('click', F.lit(1)), 'uuid', 'left') \
        .fillna(0, subset=['undel', 'open', 'click', 'resp']) \
        .select('event_uuid', 'contact_to', 'undel', 'open', 'click','event_time')\
        .distinct()

In [2]:
#join варианты
df = df_1.join(df_2, df_1.SSO_ID==df_2.SSO_ID, 'left')
df = df_1.join(df_2, "SSO_ID", 'left')

cond = [df.name == df3.name, df.age == df3.age]
df = df_1.join(df_2, cond, 'left')

cond = [df.surname == df3.surname, (df.name == df3.name) | (df.age == df3.age),
        df.orders >= df3.orders ]
df.join(df3, cond, 'outer')

cond = [((df1.col1 == df2.col2) & (df1.col3 == df2.col4))]
joined_df = df1.join(df2, on = cond, how = "inner")

In [ ]:
# удалить дубликаты
df.dropDuplicates(['name', 'height']).show()

In [ ]:
# Переименовать существующую колонку 1 на 2
df.withColumnRenamed('1','2')
# Переименовать существующую колонку .withColumnRename
df.withColumnRename("vehicle_brand","SUPER_vehicle_brand").select("SUPER_vehicle_brand").show(5)
# Создать колонку .withColumn берем vehicle_year и добавляем 1 год и получаем колонку next_year
df.withColumn("next_year",F.col("vehicle_year")+1).select("vehicle_year","next_year").show(5)

In [4]:
# создаем новую колонку 'col3' где будут колонки 'col1' без Null 
# и если 'col1' Null то взять значение из колонки 'col2'
df.withColumn('col3', F.when(F.col('col1').isNotNull(),F.col('col1')).otherwise(F.col('col2')))
# потом проверить пропуски в колонке '3'
df.where(F.col('col3').isNull()).count()